<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-직장-입력-시-위도경도-반환" data-toc-modified-id="1.-직장-입력-시-위도경도-반환-1">1. 직장 입력 시 위도경도 반환</a></span></li><li><span><a href="#2.-직장과의-거리,-지하철-가중치" data-toc-modified-id="2.-직장과의-거리,-지하철-가중치-2">2. 직장과의 거리, 지하철 가중치</a></span></li></ul></div>

In [25]:
import requests
from urllib.parse import urlparse
import urllib.request
from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas
import psycopg2   # PostgreSQL 연동 어댑터/드라이버
import json
import os
import ssl

import warnings
warnings.filterwarnings(action='ignore')

## 1. 직장 입력 시 위도경도 반환

In [22]:
# 주소 검색 함수 정의
def latlon():
    address = input('직장 주소 : ')
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['y'], document['x']]
        except:
            val = [0, 0]
    return val

In [23]:
latlon()

직장 주소 : 서울 서초구 서초1동 효령로 335


['37.4864987317089', '127.020663860591']

## 2. 직장과의 거리, 지하철 가중치

In [72]:
conn_string = "host='localhost' dbname ='house' user='postgres' password='****'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [115]:
def full(input_rent,input_deposit): 
    cur.execute(f"SELECT address, rent, deposit::integer, year::integer, station_ar, station_na, transfer FROM officetels \
                WHERE rent='{input_rent}'\
                and deposit<='{input_deposit}'") 
    row = cur.fetchall() 
    
    subway_score = []
    for i in range(len(row)):
        # 초역세권/역세권
        score = 0
        if row[i][4] == 2:
            score = 5
        elif row[i][4] == 1:
            score = 3
        elif row[i][4] == 0:
            score = 1
        
        # 환승역여부
        score2 = 0
        if row[i][6] == 1:
            score2 = 5
        elif row[i][6] == 0:
            score2 = 1
        
        subway_score.append(score+score2)
    
    row = pd.DataFrame(row, columns=['address','rent','deposit','year','station_area','station_name','transfer'])
    row['sub_score'] = subway_score
    row = row[['address','rent','deposit','year','station_name','sub_score']].sort_values(by=['sub_score'], ascending=False).reset_index(drop=True)
    
    return row.head(10)

In [116]:
def month(input_rent, input_deposit, input_pay):
    cur.execute(f"SELECT address, rent, deposit::integer, pay::integer, year::integer, station_ar, station_na, transfer FROM officetels \
                  WHERE rent='{input_rent}' and deposit <= '{input_deposit}' \
                  and pay <= '{input_pay}'") 
    row = cur.fetchall()
    
    subway_score = []
    for i in range(len(row)):
        # 초역세권/역세권
        score = 0
        if row[i][4] == 2:
            score = 10
        elif row[i][4] == 1:
            score = 5
        elif row[i][4] == 0:
            score = 0
        
        # 환승역여부
        score2 = 0
        if row[i][6] == 1:
            score2 = 5
        elif row[i][6] == 0:
            score2 = 0
        
        subway_score.append(score+score2)
    
    row = pd.DataFrame(row, columns=['address','rent','deposit','year','station_area','station_name','transfer'])
    row['sub_score'] = subway_score
    row = row[['address','rent','deposit','year','station_name','sub_score']].sort_values(by=['sub_score'], ascending=False).reset_index(drop=True)
        
    return row.head(10)  

In [117]:
def condition(input_rent):
    if input_rent =='전세':
        input_deposit = input("deposit: ")
        return full(input_rent,input_deposit)
    elif input_rent == '월세':
        input_deposit = input("deposit: ")
        input_pay = input("pay: ")
        return month(input_rent,input_deposit,input_pay)

In [118]:
input_rent = input('rent : ')
condition(input_rent)

rent : 전세
deposit: 1000


,address,rent,deposit,year,station_name,sub_score
0,서울특별시 중구 을지로6가 을지로 254,전세,1000,2004.0,['동대문역사문화공원역 2호선' '동대문역사문화공원역 4호선' '동대문역사문화공원역...,15
1,서울특별시 서초구 서초동 반포대로30길 9,전세,1000,2013.0,['교대역 3호선' '교대역 2호선' '서초역 2호선'],15
2,서울특별시 관악구 남현동 승방4길 26,전세,1000,2003.0,['사당역 2호선' '사당역 4호선'],15
3,서울특별시 서초구 서초동 반포대로30길 9,전세,1000,2013.0,['교대역 3호선' '교대역 2호선' '서초역 2호선'],15
4,서울특별시 영등포구 영등포동1가 경인로114길 12,전세,1000,2012.0,['신길역 5호선' '신길역 1호선'],15
5,서울특별시 은평구 응암동 응암로29길 18-3,전세,1000,2005.0,['응암역 6호선'],10
6,서울특별시 은평구 불광동 통일로 884,전세,500,2003.0,['연신내역 6호선' '연신내역 3호선'],10
7,서울특별시 마포구 노고산동 서강로16길 49,전세,1000,2003.0,['신촌역 2호선' '서강대역 경의선'],10
8,서울특별시 서초구 서초동 반포대로14길 36,전세,1000,1992.0,['남부터미널역 3호선'],10
9,서울특별시 금천구 독산동,전세,1000,2017.0,['청담역 7호선'],10
